# See extensionPlanReport.pdf
Find various health issues that are relevant to smoke (cardiovascular and respiratory issues).
Get national averages.
Find smoke impact to those issues and see how much it increases risk of them.
Add average for city population + percentage risk increase * leftover pop to get projected new numbers of disease in the city
Find estimate for total ED visits via relevant diseases (percentages based on number of people with specific diseases relevant to smoke).
Once we get yearly estimates do a linear regression/ARMA to project to 2050.
Project the rest of the diseases for visualizations.
Lets try to model total emergency department visits

Try to see how ED visits is proportional to number of workers in healthcare?

# Imports

In [13]:
import os, json, time, sys
from wildfire.Reader import Reader
#import geojson
import requests
#import pyproj
import math
import numpy as np
import datetime as dt
#from pyproj import Transformer, Geod
from wildfire.Reader import Reader as WFReader
from tqdm import tqdm
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import pandas as pd
import statsmodels
import statsmodels.api as sm
import csv

# Read in census data

#### Age demographics per year from 2010-2021
Codes:  
S0101_C01_001E - Total population estimate  
S0101_C01_002E - Age under 5 estimate  
S0101_C01_003E - Age 5 to 9 estimate  
S0101_C01_004E - Age 10 to 14 estimate  
S0101_C01_005E - Age 15 to 19 estimate  
S0101_C01_006E - 20 to 24  
S0101_C01_007E - 25 to 29  
S0101_C01_008E - 30 to 34  
S0101_C01_009E - 35 to 39  
S0101_C01_010E - 40 to 44  
S0101_C01_011E - 45 to 49  
S0101_C01_012E - 50 to 54  
S0101_C01_013E - 55 to 59  
S0101_C01_014E - 60 to 64  
S0101_C01_015E - 65 to 69  
S0101_C01_016E - 70 to 74  
S0101_C01_017E - 75 to 79  
S0101_C01_018E - 80 to 84  
S0101_C01_019E - 85+  

In [36]:
def getAgeInformation(filepath):
    """
    Description:
        Reads in census data age files and parses out particular fields we care about.
        See above markdown cell for code information.
    Inputs:
        filepath - String - Location of file
    Outputs:
        Dictionary
    """
    results = {}
    testdf = pd.read_csv("../Data/AgeAndSex/ACSST5Y2010.S0101-Data.csv")
    Total_pop_estimate = float(testdf['S0101_C01_001E'][1])
    results["Total_pop_estimate"] = Total_pop_estimate
    results["Pop_estimate_0to5"] = float(testdf['S0101_C01_002E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_5to9"] = float(testdf['S0101_C01_003E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_10to14"] = float(testdf['S0101_C01_004E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_15to19"] = float(testdf['S0101_C01_005E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_20to24"] = float(testdf['S0101_C01_006E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_25to29"] = float(testdf['S0101_C01_007E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_30to34"] = float(testdf['S0101_C01_008E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_35to39"] = float(testdf['S0101_C01_009E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_40to44"] = float(testdf['S0101_C01_010E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_45to49"] = float(testdf['S0101_C01_011E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_50to54"] = float(testdf['S0101_C01_012E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_55to59"] = float(testdf['S0101_C01_013E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_60to64"] = float(testdf['S0101_C01_014E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_65to69"] = float(testdf['S0101_C01_015E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_70to74"] = float(testdf['S0101_C01_016E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_75to79"] = float(testdf['S0101_C01_017E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_80to84"] = float(testdf['S0101_C01_018E'][1]) * Total_pop_estimate/100
    results["Pop_estimate_85toINF"] = float(testdf['S0101_C01_019E'][1]) * Total_pop_estimate/100
    return results

Ages_2010 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2010.S0101-Data.csv")
Ages_2011 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2011.S0101-Data.csv")
Ages_2012 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2012.S0101-Data.csv")
Ages_2013 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2013.S0101-Data.csv")
Ages_2014 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2014.S0101-Data.csv")
Ages_2015 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2015.S0101-Data.csv")
Ages_2016 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2016.S0101-Data.csv")
Ages_2017 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2017.S0101-Data.csv")
Ages_2018 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2018.S0101-Data.csv")
Ages_2019 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2019.S0101-Data.csv")
Ages_2020 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2020.S0101-Data.csv")
Ages_2021 = getAgeInformation("../Data/AgeAndSex/ACSST5Y2021.S0101-Data.csv")

#### Industry by citizen over 16

# Basic visualizations